In [1]:
import logging
import pandas as pd

from examples.test_data_initialization import example_sensor_data, get_test_files

## Load configuration for training & testing

In [2]:
from basys4ipps_ifw_agent.agent.basys_agent import BasysAgent, BasysConfig
from pathlib import Path

basys_config = BasysConfig.load(create_if_none=True, path=Path("../config.yaml"))
basys_config.alpha_safety_factor = 0.999
basys_config.tsfresh_features = Path(*Path(basys_config.tsfresh_features).parts[1:]).as_posix()
basys_config.tsfresh_random_forest = Path(*Path(basys_config.tsfresh_random_forest).parts[1:]).as_posix()
basys_config.use_tsfresh_features = True

Reading config file 'C:/git/BaSys4IPPS_IFW_Agent/config.yaml'


# Prepare training data

In [3]:
all_files = get_test_files("./train" )

training_index = list(range(20))

x_train, x_test = example_sensor_data(all_files, training_index, test_index=115, sensor=5,
                                      basys_config=basys_config)

Found 14 files!


# Train & Test

In [4]:
agent = BasysAgent(basys_config)

logging.info("Start training")
agent.fit(x_train, basys_config)

logging.info("Start testing")
score_prob, alarm, exp_downtime = agent.predict(x_test, basys_config)

INFO:BasysLogger:Extract features from training data


Using tsfresh features from  tsfresh_features.json


Feature Extraction: 100%|██████████| 20/20 [00:02<00:00,  6.77it/s]
INFO:BasysLogger:Fit the outlier detection model
INFO:BasysLogger:Extract features from test data


Using tsfresh features from  tsfresh_features.json


Feature Extraction: 100%|██████████| 1/1 [00:02<00:00,  2.76s/it]
INFO:BasysLogger:Execute decision function
INFO:BasysLogger:safety_factor=0.999000,
score_prob=[1.]
alarm=[ True]
exp_downtime=1
